# Statistical Assertions

In the lecture we have discussed about different research in testing quantum programs. This notebook focusses on some research based on statisitcal assertions than can be used to test quantum cirquits during the development process. The notebook mainly follows some of the ideas suggested in [2].

In [2] six different types of error sources during the development process and suggested defenses are addressed. In this notebook only three of them will be explored / experimented with:
* assert if a qubit is in a classical state
* assert if a qubit is in a superposition
* assert if two qubits are entangled

General idea in [2]: use statistical tests

Concrete: in [2] they propose to use the chi squared test:
$ X^2 = \frac{(observed - expected)^2}{expected}$


* [1] Zhao (2020): "Quantum Software Engineering - Landscapes and Horizons"
* [2] Huang, Martonosi(2019): "Statistical Assertions for Validating Patterns and Finding Bugs in Quantum Programs"

In [1]:
import cirq

## Helper classes

* StatisticalAssertion: class that implements the three mentioned assert methods and some additional helper methods
* Simulation: implements a minimalistic simulate method

Comment: the SciPy library implements some helpful statistical functions and variables => it makes sense to integrate the SciPy library

In [2]:
class StatisticalAssertion:    

    def chi_squared_test(self, observed, expected, alpha, runs):
        is_rejected = False

        if not expected == 0:
            chi2 = (observed - expected) ** 2 / expected
        else:
            chi2 = 0
        
        critical_value = 1 - (alpha * runs) / runs
        
        if chi2 > critical_value:
            is_rejected = True

        return is_rejected
    
    
    def get_expected_classical(self, runs, state_zero=True):
        if state_zero:
            return [runs, 0]
        else:
            return [0, runs]
        
    
    def get_expected_superposition(self, runs):
        return [runs/2, runs/2]
   

    def get_expected_entanglement(self, runs, observed):
        expected = observed
        expected[0] = (observed[0] + observed[2]) * (observed[0] + observed[1]) / runs
        expected[1] = (observed[1] + observed[3]) * (observed[0] + observed[1]) / runs
        expected[2] = (observed[0] + observed[2]) * (observed[2] + observed[3]) / runs
        expected[3] = (observed[1] + observed[3]) * (observed[2] + observed[3]) / runs
        return expected

    
    def assert_classical(self, observed, runs, state_zero=True, alpha=0.05):
        expected = self.get_expected_classical(runs, state_zero)

        if state_zero:
            is_rejected = self.chi_squared_test(observed[0], expected[0], alpha, runs)
        else:
            is_rejected = self.chi_squared_test(observed[1], expected[1], alpha, runs)

        if is_rejected:
            return False
        else:
            return True
        
    
    def assert_superposition(self, observed, runs, alpha=0.05):
        expected = self.get_expected_superposition(runs)
        is_rejected_0 = self.chi_squared_test(observed[0], expected[0], alpha, runs)
        is_rejected_1 = self.chi_squared_test(observed[1], expected[1], alpha, runs)
        
        if ((not is_rejected_0) and (not is_rejected_1)):
            return True
        else:
            return False

        
    def assert_entanglement(self, observed, runs, alpha=0.05):
        expected = self.get_expected_entanglement(runs, observed)
        print("assert_entanglement", expected)
        
        is_rejected_0 = self.chi_squared_test(observed[0], expected[0], alpha, runs)
        is_rejected_1 = self.chi_squared_test(observed[3], expected[3], alpha, runs)
               
        if ((not is_rejected_0) and (not is_rejected_1)):
            return True
        else:
            return False



class Simulation:
    def simulate(self, circuit, repetitions, measurement_key):
        result = {}
        
        simulator = cirq.Simulator()
        results = simulator.simulate(circuit)
    
        samples = simulator.run(circuit, repetitions=repetitions)
        result = samples.histogram(key=measurement_key)

        return result

### Test the Helper classes
The following cell prints some examples of the chi squared test. 

It also tests the helper functions to get the expected values.
* classical state: we expect a 1x2-matrix where the given runs are distributed to the classical state
* superposition: we expect a 1x2-matrix where the given runs are equaly distributed to both states
* entanglement:
  * in case of a classical state: we expect a 2x2 matrix where the given runs are distributed to |00> or |11>
  * in case of superposition: we expect a 2x2 matrix where the given runs are equaly distributed to to |00> and |11>

Comment: in get_expected_entanglement the distribution for superposition is not correct (should be [250.0, 0, 0, 250.0])

In [3]:
assertion = StatisticalAssertion()
# chi squared test method
print("H_0 rejected:", assertion.chi_squared_test(1000, 1000, 0.05, 1000))
print("H_0 rejected:", assertion.chi_squared_test(970, 1000, 0.05, 1000))
print("H_0 accepted:", assertion.chi_squared_test(969, 1000, 0.05, 1000))

# get expected classical
print("Classical state zero (run 1000):", assertion.get_expected_classical(1000, True))
print("Classical state one (run 1000):", assertion.get_expected_classical(1000, False))

# get expected superposition
print("Superposition (run 1000):", assertion.get_expected_superposition(1000))
print("Superposition (run 500):", assertion.get_expected_superposition(500))

# get expected entanglement
print("entanglement (run 1000):", assertion.get_expected_entanglement(1000, [1000, 0, 0, 0]))
print("entanglement (run 500):", assertion.get_expected_entanglement(1000, [500, 0, 0, 500]))

H_0 rejected: False
H_0 rejected: False
H_0 accepted: True
Classical state zero (run 1000): [1000, 0]
Classical state one (run 1000): [0, 1000]
Superposition (run 1000): [500.0, 500.0]
Superposition (run 500): [250.0, 250.0]
entanglement (run 1000): [1000.0, 0.0, 0.0, 0.0]
entanglement (run 500): [250.0, 125.0, 125.0, 390.625]


## Some basic test cases

### Test classical states
We want to check if the assert_classical method can recognize correctly, if:

* a qubit is in a given classical state
* a qubit is not in a given classical state

Therefore, two simple test_circuits will be created:
* circuit_classical_zero: circuit with one qubit in state zero and a measurement
* circuit_classical_one: circuit with one qubit in state one and a measurement

In [4]:
circuit_classical_zero = cirq.Circuit()
circuit_classical_zero.append(cirq.measure(cirq.NamedQubit('q0'), key='result'))
print('circuit_classical_zero:', circuit_classical_zero)

circuit_classical_one = cirq.Circuit()
circuit_classical_one.append(cirq.X(cirq.NamedQubit('q0')))
circuit_classical_one.append(cirq.measure(cirq.NamedQubit('q0'), key='result'))
print('circuit_classical_one:', circuit_classical_one)

circuit_classical_zero: q0: ───M('result')───
circuit_classical_one: q0: ───X───M('result')───


In [5]:
runs = 1000
simulation = Simulation()
assertion = StatisticalAssertion()
measurement_classical_zero = simulation.simulate(circuit_classical_zero, runs, 'result')
print('measurement_classical_zero: state is |0>?', assertion.assert_classical(measurement_classical_zero, runs, True))
print('measurement_classical_zero: state is |1>?', assertion.assert_classical(measurement_classical_zero, runs, False))

measurement_classical_one = simulation.simulate(circuit_classical_one, runs, 'result')
print('measurement_classical_one: state is |0>?', assertion.assert_classical(measurement_classical_one, runs, True))
print('measurement_classical_one: state is |1>?', assertion.assert_classical(measurement_classical_one, runs, False))

measurement_classical_zero: state is |0>? True
measurement_classical_zero: state is |1>? False
measurement_classical_one: state is |0>? False
measurement_classical_one: state is |1>? True


### Test superposition
We want to check if the assert_superposition method can recognize correctly, if:

* a qubit is in a superposition
* a qubit is not in a superposition

Therefore, the two previous test_circuits will be reused for testing if a qubit is not in superposition. In addition a third test_circuit is created:
* circuit_superposition: circuit with one qubit in superposition and a measurement

In [6]:
runs = 1000
simulation = Simulation()
assertion = StatisticalAssertion()

measurement_classical_zero = simulation.simulate(circuit_classical_zero, runs, 'result')
print('measurement_classical_zero: state is in superosition?', assertion.assert_superposition(measurement_classical_zero, runs))

measurement_classical_one = simulation.simulate(circuit_classical_one, runs, 'result')
print('measurement_classical_one: state is in superposition?', assertion.assert_superposition(measurement_classical_one, runs))

print('*** superposition test***')
circuit_superposition = cirq.Circuit()
circuit_superposition.append(cirq.H(cirq.NamedQubit('q0')))
circuit_superposition.append(cirq.measure(cirq.NamedQubit('q0'), key='result'))
print('circuit_superposition:', circuit_superposition)

measurement_superosition = simulation.simulate(circuit_superposition, runs, 'result')
print('measurement_superosition: state is in superposition?', assertion.assert_superposition(measurement_superosition, runs))


measurement_classical_zero: state is in superosition? False
measurement_classical_one: state is in superposition? False
*** superposition test***
circuit_superposition: q0: ───H───M('result')───
measurement_superosition: state is in superposition? True


### Test entanglement
We want to check if the assert_entanglement method can recognize correctly, if:

* two qubits are entangled
* two qubits are not entangled

Therefore, we can reuse the already created single qubit test circuits => we expect them to be recognized as "not entangled" since there is only one qubit.

We will create the following additional two qubit test circuits:

In [17]:
runs = 1000
simulation = Simulation()
assertion = StatisticalAssertion()


# positive tests (|00> and |11>)
print("-------positive tests (|00> and |11>)--------")
circuit_entanglement_zero = cirq.Circuit()
circuit_entanglement_zero.append(cirq.CX(cirq.NamedQubit('q0'), cirq.NamedQubit('q1')))
circuit_entanglement_zero.append(cirq.measure(cirq.NamedQubit('q0'), cirq.NamedQubit('q1'), key='result'))
print('\ncircuit_entanglement_zero:')
print(circuit_entanglement_zero)

measurement_entanglement_zero = simulation.simulate(circuit_entanglement_zero, runs, 'result')
print('circuit_entanglement_zero: state is entangled?', assertion.assert_entanglement(measurement_entanglement_zero, runs))

circuit_entanglement_one = cirq.Circuit()
circuit_entanglement_one.append(cirq.X(cirq.NamedQubit('q0')))
circuit_entanglement_one.append(cirq.CX(cirq.NamedQubit('q0'), cirq.NamedQubit('q1')))
circuit_entanglement_one.append(cirq.measure(cirq.NamedQubit('q0'), cirq.NamedQubit('q1'), key='result'))
print('\ncircuit_entanglement_one:')
print(circuit_entanglement_one)

measurement_entanglement_one = simulation.simulate(circuit_entanglement_one, runs, 'result')
print('circuit_entanglement_one: state is entangled?', assertion.assert_entanglement(measurement_entanglement_one, runs))


# positive tests (superposition)
print("\n-------positive tests (superposition)--------")
circuit_entanglement_superposition = cirq.Circuit()
q0, q1 = cirq.NamedQubit.range(0, 2, prefix='q')
circuit_entanglement_superposition.append(cirq.H(q0))
circuit_entanglement_superposition.append(cirq.CX(q0, q1))
circuit_entanglement_superposition.append(cirq.measure(q0, q1, key='result'))
print('\ncircuit_entanglement_superposition:')
print(circuit_entanglement_superposition)

measurement_entanglement_superposition = simulation.simulate(circuit_entanglement_superposition, runs, 'result')
print('circuit_entanglement_superposition: state is entangled?', assertion.assert_entanglement(measurement_entanglement_superposition, runs))


#negative tests (only one qubit at all)
print("\n-------negative tests (only one qubit at all)--------")
print(circuit_classical_zero)
measurement_classical_zero = simulation.simulate(circuit_classical_zero, runs, 'result')
print('\ncircuit_classical_zero: state is entangled?', assertion.assert_entanglement(measurement_classical_zero, runs))

print(circuit_classical_one)
measurement_classical_one = simulation.simulate(circuit_classical_one, runs, 'result')
print('\ncircuit_classical_one: state is entangled?', assertion.assert_entanglement(measurement_classical_one, runs))

print(circuit_superposition)
measurement_superposition = simulation.simulate(circuit_superposition, runs, 'result')
print('\ncircuit_superposition: state is entangled?', assertion.assert_entanglement(measurement_superposition, runs))


# negative tests (|01> and |10>)
print("\n-------negative tests (|01> and |10>)--------")
circuit_zero_one = cirq.Circuit()
circuit_zero_one.append(cirq.X(cirq.NamedQubit('q1')))
circuit_zero_one.append(cirq.measure(cirq.NamedQubit('q0'), cirq.NamedQubit('q1'), key='result'))
print('\ncircuit_zero_one:')
print(circuit_zero_one)
measurement_zero_one = simulation.simulate(circuit_zero_one, runs, 'result')
print('\circuit_zero_one: state is entangled?', assertion.assert_entanglement(measurement_zero_one, runs))


circuit_one_zero = cirq.Circuit()
circuit_one_zero.append(cirq.X(cirq.NamedQubit('q0')))
circuit_one_zero.append(cirq.measure(cirq.NamedQubit('q0'), cirq.NamedQubit('q1'), key='result'))
print('\ncircuit_one_zero:')
print(circuit_one_zero)

measurement_one_zero = simulation.simulate(circuit_one_zero, runs, 'result')
print('\circuit_one_zero: state is entangled?', assertion.assert_entanglement(measurement_one_zero, runs))


-------positive tests (|00> and |11>)--------

circuit_entanglement_zero:
q0: ───@───M('result')───
       │   │
q1: ───X───M─────────────
assert_entanglement Counter({0: 1000.0, 1: 0.0, 2: 0.0, 3: 0.0})
circuit_entanglement_zero: state is entangled? True

circuit_entanglement_one:
q0: ───X───@───M('result')───
           │   │
q1: ───────X───M─────────────
assert_entanglement Counter({3: 1000.0, 0: 0.0, 1: 0.0, 2: 0.0})
circuit_entanglement_one: state is entangled? True

-------positive tests (superposition)--------

circuit_entanglement_superposition:
q0: ───H───@───M('result')───
           │   │
q1: ───────X───M─────────────
assert_entanglement Counter({3: 361.4891876784046, 0: 281.961, 1: 132.239709, 2: 132.239709})
circuit_entanglement_superposition: state is entangled? True

-------negative tests (only one qubit at all)--------
q0: ───M('result')───
assert_entanglement Counter({0: 1000.0, 1: 0.0, 2: 0.0, 3: 0.0})

circuit_classical_zero: state is entangled? True
q0: ───X───M('re

## Next Steps: Test the QPE algorithm

So far we only run simple test cases and always measured directly the qubit(s) under test.

Next step is to try the statisitcal assertions on a more complex circuit, e.g. the Quantum Phase Estimation from homework 02.

In [2] the shor algorithm was tested. This already gives some suggestions where to put the assertions. In [1] is suggested, instead of directly measuring the qubits under test, some additional anchilla qubit(s) are used and entangled with the qubit under test. Then the anchilla qubit is measured instead to avoid state collapse on the qubits under test.

* use ancilla qubit to measure the assertions
* assert after initalization, before iQFT

## Feedback

* Further reading / exploring: does a measurement on one of the entangled qubits effect both qubits?
* Test entanglement: 
  * how to calculate the expected values for the chi test dynamically and correctly?
  * how to make sure, the assert function does not detect an entanglement for two qubits being "close" but not entangled?
  * how to addapt the accepted error (/ p-value / critical value) to the number of runs => the more runs we have the smaller the accepted error should be?
 
* how to detected / deal with accumulated errors during runtime?

* the statistical assertions are deisgned to use during development but not during / after deployment => how to test circuits there?

### Additional Notes

The findings from [2] are implemented in Q# and Quiskit => maybe look into code base over there and learn from them.

## Related Work

Some of the following findings we discussed during lecture or are mentioned in [1]:

* "An Introduction to Quantum Error Correction and Fault-Tolerant Quantum Computation" (arXiv: 0904.2557)
* "Secure quantum conversation through non-destructive discrimination of highly entangled multipartite states" (arXiv: 0906.4323)
* "Poq: Projection-based Runtime Assertions for Debugging on a Quantum Computer" (arXiv: 911.12855v1)
* "Debugging Quantum Processes Using Monitoring Measurements" (arXiv: 1403.4344)
* "Quantum Circuits for Dynamic Runtime Assertions in Quantum Computation" (Liu et. al, 2020)
* Fuzzing